In [4]:
import json
import geopandas as gpd
import plotly.express as px

with open("iraq_provinces.geojson") as f:
    provinces = json.load(f)

# Convert to a GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(provinces["features"])

# Assuming 'NAME_1' field is the province name in your geojson
fig = px.choropleth(gdf, geojson=provinces, locations='NAME_1',
                           color='NAME_1',  # or any other field you want to color by
                           color_continuous_scale="Viridis",
                           scope="asia",  # Changed to 'asia' from 'usa'
                           labels={'NAME_1':'Province Name'}  # update labels as per your data
                          )
fig.update_geos(showcountries=False, showcoastlines=True, showland=True, fitbounds="locations")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


AttributeError: 'str' object has no attribute 'read'

In [13]:
from shapely.geometry import shape, mapping
from shapely.ops import unary_union
import geojson

def merge_provinces(geojson_file, province_names):
    with open(geojson_file, 'r') as f:
        data = geojson.load(f)

    # Filter out the provinces to merge
    provinces_to_merge = [feature for feature in data['features'] if feature['properties']['NAME_1'] in province_names]

    # Convert the GeoJSON features to shapely geometry objects
    shapes = [shape(feature['geometry']) for feature in provinces_to_merge]

    # Merge the provinces together
    merged = unary_union(shapes)

    # Convert the shapely object back to a GeoJSON feature
    merged_geojson = geojson.Feature(geometry=mapping(merged), properties={"NAME_1": "Merged Area"})

    return merged_geojson

def write_to_geojson(merged_provinces, output_file):
    # Create a new GeoJSON FeatureCollection with all merged provinces
    feature_collection = geojson.FeatureCollection(merged_provinces)

    # Write the FeatureCollection to a new file
    with open(output_file, 'w') as f:
        geojson.dump(feature_collection, f)


In [14]:
province_groups = [
    ("MNF-W", ["Al-Anbar"]),
    ("MND-N", ["Ninawa", "Dihok", "Arbil"]),
    ("MND-C", ["Karbala", "Wasit", "Al-Qadisiyah", "Al-Najaf", "Babil"]),
    ("MND-SE", ["Al-Basrah", "Maysan", "Al-Muthannia", "Dhi-Qar"]),
    ("MND-Baghdad", ["Baghdad"]),
    ("MND-NE", ["At-Ta'mim", "Sala ad-Din", "Diyala", "As-Sulaymaniyah"]),
]

merged_provinces = []

for new_name, province_names in province_groups:
    merged_province = merge_provinces("iraq_provinces.geojson", province_names)
    merged_province["properties"]["NAME_1"] = new_name
    merged_provinces.append(merged_province)

write_to_geojson(merged_provinces, "merged_provinces.geojson")
